In [ ]:
# ! pip install micrograd==0.1.0 scikit-learn>=0.20 optuna

###  Kittylyst demo

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons, make_blobs
%matplotlib inline

In [ ]:
from micrograd.engine import Value
from micrograd.nn import Neuron, Layer, MLP

In [ ]:
# from kittylyst import (
#     MicroLoader, MicroCriterion, MicroOptimizer, MicroScheduler,
#     SingleStageExperiment, IRunner,SupervisedRunner, ICallback,
#     CriterionCallback, AccuracyCallback, 
#     OptimizerCallback, SchedulerCallback,
#     LoggerCallback
# )

In [ ]:
from typing import *
from kittylyst import *

In [ ]:
# make up a dataset
def make_dataset(seed=42, n_samples=100):
    np.random.seed(seed)
    random.seed(seed)
    X, y = make_moons(n_samples=100, noise=0.1)

    y = y*2 - 1 # make y be -1 or 1
    return X, y

def visualize_dataset(X, y):
    plt.figure(figsize=(5,5))
    plt.scatter(X[:,0], X[:,1], c=y, s=20, cmap='jet')

# let's create train data
X_train, y_train = make_dataset()
visualize_dataset(X_train, y_train)

In [ ]:
# valid data
X_valid, y_valid = make_dataset(seed=137)
visualize_dataset(X_valid, y_valid)

In [ ]:
# and another train one (why not?)
X_train2, y_train2 = make_dataset(seed=1337)
visualize_dataset(X_train2, y_train2)

In [ ]:
# initialize a model 
model = MLP(2, [16, 16, 1]) # 2-layer neural network
print(model)
print("number of parameters", len(model.parameters()))

In [ ]:
def visualize_decision_boundary(X, y, model):
    h = 0.25
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Xmesh = np.c_[xx.ravel(), yy.ravel()]
    inputs = [list(map(Value, xrow)) for xrow in Xmesh]
    scores = list(map(model, inputs))
    Z = np.array([s.data > 0 for s in scores])
    Z = Z.reshape(xx.shape)

    fig = plt.figure()
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.8)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.show()
    return fig

In [ ]:
_ = visualize_decision_boundary(X_valid, y_valid, model)

In [ ]:
loaders = {
    "train_1": MicroLoader(X_train, y_train), 
    "train_2": MicroLoader(X_train2, y_train2), 
    "valid": MicroLoader(X_valid, y_valid), 
}

---

### Act 1 - ``CustomRunner`` solution

Suppose you have your favorite `for-loop` pipeline and want to get rid of `for-for-for` stuff to make code more clean. In this case you can define your own ``CustomRunner`` with ``_handle_batch`` method and just run it.

Whole ``Runner`` source code is located [here](github.com/Scitator/kittylyst/blob/master/kittylyst/runner.py) (~100 lines of code).

In [ ]:
model = MLP(2, [16, 16, 1])
experiment = SingleStageExperiment(model=model, loaders=loaders, num_epochs=10)

class CustomRunner(IStageBasedRunner):
    def _handle_batch(self, batch):
        features, targets = batch
        scores = list(map(self.model, features))
        
        losses = [(1 + -yi*scorei).relu() for yi, scorei in zip(targets, scores)]
        data_loss = sum(losses) * (1.0 / len(losses))
        # L2 regularization
        alpha = 1e-4
        reg_loss = alpha * sum((p*p for p in self.model.parameters()))
        total_loss = data_loss + reg_loss

        # also get accuracy
        accuracy = [(yi > 0) == (scorei.data > 0) for yi, scorei in zip(targets, scores)]
        accuracy = sum(accuracy) / len(accuracy)
        
        print(f"{self.loader_key}: accuracy {accuracy}, loss {total_loss.data}")
        
        if self.is_train_loader:
            # backward
            self.model.zero_grad()
            total_loss.backward()

            # update (sgd)
            learning_rate = 1.0 - 0.9 * self.stage_epoch_step / self.stage_epoch_len
            for p in self.model.parameters():
                p.data -= learning_rate * p.grad

CustomRunner().run(experiment)

In [ ]:
_ = visualize_decision_boundary(X_valid, y_valid, model)

---

### Act 2 - ``SupervisedRunner`` solution

Let's make a bit more abstract. Let's introduce ``SupervisedRunner``, that knows how to execute Supervised models and ``CriterionCallback/OptimizerCallback/SchedulerCallback`` for typical criterions/optimizers/schedulers steps. 

Additionaly let's wrap accuracy with ``AccuracyCallback``, as far as metrics are general too. 

Finally, let's make our logs looks consistent with ``LoggerCallback``.

Whole ``Callback`` source code is located [here](github.com/Scitator/kittylyst/blob/master/kittylyst/callback.py) (~100 lines of code).

In [ ]:
model = MLP(2, [16, 16, 1])
criterion = MicroCriterion()
optimizer = MicroOptimizer(model)
scheduler = MicroScheduler(optimizer, num_epochs=10)
experiment = SingleStageExperiment(
    model=model, 
    criterion=criterion, 
    optimizer=optimizer, 
    scheduler=scheduler,
    loaders=loaders, 
    num_epochs=6,
    callbacks={
        "criterion": CriterionCallback(), 
        "accuracy": MetricCallback(
            metric=AccuracyMetric(), compute_on_batch=True,
            input_key="targets", output_key="logits"
        ),
        "auc": MetricCallback(
            metric=AUCMetric(), compute_on_batch=False,
            input_key="targets", output_key="logits"
        ), 
        "optimizer": OptimizerCallback(), 
        "scheduler": SchedulerCallback(),
        "checkpoint1": CheckpointCallback(# TopNMetricHandlerCallback(
            loader_key="valid", metric_key="auc", 
            minimize=False, save_n_best=3
        ),
        "checkpoint2": CheckpointCallback(# TopNMetricHandlerCallback(
            loader_key="valid", metric_key="loss_mean", 
            minimize=True, save_n_best=1
        ),
        "verbose": VerboseCallback(),
    },
    loggers={
        "console": ConsoleLogger(),
        "csv": LogdirLogger(logdir="./logdir"),
    }
)

SupervisedRunner().run(experiment)

In [ ]:
_ = visualize_decision_boundary(X_valid, y_valid, model)

---

### Act 3 - ``CustomExperiment`` solution - multi-stage experiment with ``VisualizationCallback``

Let's go even harder. Suppose we want to firstly train our model on `train_1` data, and only after that - on `train_2`. This case could be easily handled with ``CustomExperiment``, where you could redefine your experiment components for each stage of your experiment.

Whole ``Experiemnt`` source code is located [here](github.com/Scitator/kittylyst/blob/master/kittylyst/experiment.py) (~100 lines of code).

In [ ]:
import io
import cv2
import numpy as np
import matplotlib.pyplot as plt

def get_img_from_fig(fig, dpi=180):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=dpi)
    buf.seek(0)
    img_arr = np.frombuffer(buf.getvalue(), dtype=np.uint8)
    buf.close()
    img = cv2.imdecode(img_arr, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [ ]:
class CustomExperiment(SingleStageExperiment):
    @property
    def stages(self):
        return self._stage

    def get_data(self, stage: str):
        return self._loaders[stage]

    
class VisualizationCallback(ICallback):
    def on_epoch_end(self, runner):
        img = visualize_decision_boundary(X_valid, y_valid, runner.model)
        img = get_img_from_fig(img)
        runner.log_image(img, scope="epoch")


loaders = {
    "stage_1": {
        "train_1": MicroLoader(X_train, y_train), 
        "valid": MicroLoader(X_valid, y_valid), 
    },
    "stage_2": {
        "train_2": MicroLoader(X_train2, y_train2), 
        "valid": MicroLoader(X_valid, y_valid), 
    },
}
stages = loaders.keys()

model = MLP(2, [16, 16, 1])
criterion = MicroCriterion()
optimizer = MicroOptimizer(model)
scheduler = MicroScheduler(optimizer, num_epochs=10)
experiment = CustomExperiment(
    model=model, 
    criterion=criterion, 
    optimizer=optimizer, 
    scheduler=scheduler,
    loaders=loaders, 
    num_epochs=4,
    callbacks={
        "criterion": CriterionCallback(), 
        "accuracy": MetricCallback(
            metric=AccuracyMetric(), compute_on_batch=True,
            input_key="targets", output_key="logits"
        ),
        "auc": MetricCallback(
            metric=AUCMetric(), compute_on_batch=False,
            input_key="targets", output_key="logits"
        ), 
        "optimizer": OptimizerCallback(), 
        "scheduler": SchedulerCallback(),
            
        "checkpoint": CheckpointCallback(# TopNMetricHandlerCallback(
            loader_key="valid", metric_key="auc", 
            minimize=False, save_n_best=3
        ),
        "visualization": VisualizationCallback(),
#         "verbose": VerboseCallback(),
        
    },
    loggers={
#         "console": ConsoleLogger(exclude=["batch", "global_batch", "loader", "global_epoch"]),
        "console": ConsoleLogger(),
        "csv": LogdirLogger(logdir="./logdir2"),
    },
    stage=stages  # <-- here is the trick for multi-stage support
)

SupervisedRunner().run(experiment)

In [ ]:
_ = visualize_decision_boundary(X_valid, y_valid, model)

### Act 4 - integration with hyperparameter search

In [ ]:
from datetime import datetime
import optuna    

def objective(trial):
    num_epochs = 2
    l2_alpha = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    num_hidden1 = int(trial.suggest_loguniform("num_hidden1", 2, 16))
    num_hidden2 = int(trial.suggest_loguniform("num_hidden2", 2, 16))
    
    loaders = {
        "train_1": MicroLoader(X_train, y_train), 
        "train_2": MicroLoader(X_train2, y_train2), 
        "valid": MicroLoader(X_valid, y_valid), 
    }

    model = MLP(2, [num_hidden1, num_hidden2, 1])
    criterion = MicroCriterion()
    optimizer = MicroOptimizer(model)
    scheduler = MicroScheduler(optimizer, num_epochs=num_epochs)

    experiment = SingleStageExperiment(
        model=model, 
        criterion=criterion, 
        optimizer=optimizer, 
        scheduler=scheduler,
        loaders=loaders, 
        num_epochs=num_epochs,
        callbacks={
            "criterion": CriterionCallback(alpha=l2_alpha), 
            "accuracy": MetricCallback(
                metric=AccuracyMetric(), compute_on_batch=True,
                input_key="targets", output_key="logits"
            ),
            "auc": MetricCallback(
                metric=AUCMetric(), compute_on_batch=False,
                input_key="targets", output_key="logits"
            ), 
            "optimizer": OptimizerCallback(), 
            "scheduler": SchedulerCallback(),
    #         "verbose": VerboseCallback(),
            "checkpoint": CheckpointCallback(# TopNMetricHandlerCallback(
                loader_key="valid", metric_key="auc", 
                minimize=False, save_n_best=1,
            ),
            "optuna": OptunaPruningCallback(loader_key="valid", metric_key="auc")
        },
        loggers={
            "console": ConsoleLogger(),
            "csv": LogdirLogger(logdir=f"./logdir/{datetime.now().strftime('%Y%m%d-%H%M%S')}"),
        },
        trial=trial,
#         hparams={"l2_alpha": l2_alpha, "num_hidden1": num_hidden1, "num_hidden2": num_hidden2},
    )

    runner = SupervisedRunner()
    runner.run(experiment)
    score = runner.callbacks["checkpoint"].top_best_metrics[0][0]
    
    return score

study = optuna.create_study(
    direction="maximize",
#     direction="minimize",
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=0, n_warmup_steps=0, interval_steps=1
    ),
)
study.optimize(objective, n_trials=5, timeout=300)
print(study.best_value, study.best_params)

### Act 5 - fully custom experiment

In [ ]:
class CustomExperiment(IExperiment):
    @property
    def seed(self) -> int:
        return 73

    @property
    def name(self) -> str:
        # @TODO: auto-generate name?
        return "experiment73"

    @property
    def hparams(self) -> Dict:
        return {}

    @property
    def stages(self) -> List[str]:
        return ["stage_1", "stage_2"]

    def get_stage_params(self, stage: str) -> Dict[str, Any]:
        if stage == "stage_1":
            return {
                "num_epochs": 10,
                "migrate_from_previous_stage": False,
            }
        elif stage == "stage_2":
            return {
                "num_epochs": 6,
                "migrate_from_previous_stage": False,
            }
        else:
            raise NotImplemented()

    def get_data(self, stage: str) -> Dict[str, Any]:
        if stage == "stage_1":
            return {
                "train_1": MicroLoader(X_train, y_train), 
                "valid": MicroLoader(X_valid, y_valid), 
            }
        elif stage == "stage_2":
            return {
                "train_2": MicroLoader(X_train2, y_train2), 
                "valid": MicroLoader(X_valid, y_valid), 
            }
        else:
            raise NotImplemented()

    def get_model(self, stage: str):
        return MLP(2, [16, 16, 1])

    def get_criterion(self, stage: str):
        return MicroCriterion()

    def get_optimizer(self, stage: str, model):
        if stage == "stage_1":
            return MicroOptimizer(model, lr=1e-3)
        elif stage == "stage_2":
            return MicroOptimizer(model, lr=1e-4)
        else:
            raise NotImplemented()

    def get_scheduler(self, stage: str, optimizer):
        if stage == "stage_1":
            return MicroScheduler(optimizer, num_epochs=10)
        elif stage == "stage_2":
            return MicroScheduler(optimizer, num_epochs=6)
        else:
            raise NotImplemented()
        

    def get_callbacks(self, stage: str) -> Dict[str, ICallback]:
        return {
            "criterion": CriterionCallback(), 
            "accuracy": MetricCallback(
                metric=AccuracyMetric(), compute_on_batch=True,
                input_key="targets", output_key="logits"
            ),
            "auc": MetricCallback(
                metric=AUCMetric(), compute_on_batch=False,
                input_key="targets", output_key="logits"
            ), 
            "optimizer": OptimizerCallback(), 
            "scheduler": SchedulerCallback(),
            "checkpoint1": CheckpointCallback(# TopNMetricHandlerCallback(
                loader_key="valid", metric_key="auc", 
                minimize=False, save_n_best=3
            ),
            "checkpoint2": CheckpointCallback(# TopNMetricHandlerCallback(
                loader_key="valid", metric_key="loss_mean", 
                minimize=True, save_n_best=1
            ),
            "verbose": VerboseCallback(),
        }

    def get_engine(self) -> IEngine:
        return Engine()

    def get_trial(self) -> ITrial:
        return Trial()

    def get_loggers(self) -> Dict[str, ILogger]:
        return {
            "console": ConsoleLogger(),
            "csv": LogdirLogger(logdir="./logdir5"),
        }


experiment = CustomExperiment()
SupervisedRunner().run(experiment)

---

🎉 You have passed ``Kittylyst`` tutorial! This is just a minimal educational demo, but I hope you found it interesting for your deep learning research code organisation.

For more advanced and production-ready solution please follow our [Catalyst](https://github.com/catalyst-team/catalyst) repository.

PS. If you are interested in deep learning you could also try out our [dl-course](https://github.com/catalyst-team/dl-course).